<a href="https://colab.research.google.com/github/gabegomes/orca/blob/master/GAP_train_ORCA_xtb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q condacolab imolecule 
import condacolab
# condacolab.install_from_url("https://cmu.box.com/shared/static/70o345tyghhfdx7fufirbrz5ljzli2p2.sh")
condacolab.install_from_url("file:///content/drive/MyDrive/DMDS_CMU_Spring_2022/DMDS_CMU_Spring_2022_ConstructorForCondaColab/condacolab_DMDS_CMU_py37-0.1-Linux-x86_64.sh")

     |████████████████████████████████| 475 kB 9.3 MB/s 
⏬ Downloading file:///content/drive/MyDrive/DMDS_CMU_Spring_2022/DMDS_CMU_Spring_2022_ConstructorForCondaColab/condacolab_DMDS_CMU_py37-0.1-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:02:49
🔁 Restarting kernel...


In [ ]:
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


In [ ]:
import sys
import os
import subprocess
from os import path
import multiprocessing
import multiprocessing.pool
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem.Draw.MolDrawing import MolDrawing
from rdkit.Chem.Draw import IPythonConsole

In [ ]:
! pip install cctk selfies presto-md quippy-ase py3Dmol
! jupyter-nbextension enable nglview --py --sys-prefix

     |████████████████████████████████| 98 kB 5.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 29.9 MB 1.2 MB/s 
     |████████████████████████████████| 321 kB 70.1 MB/s 
     |████████████████████████████████| 32.2 MB 67.3 MB/s 
     |████████████████████████████████| 596 kB 80.0 MB/s 
     |████████████████████████████████| 4.1 MB 73.5 MB/s 
     |████████████████████████████████| 1.2 MB 84.0 MB/s 
     |████████████████████████████████| 69 kB 9.7 MB/s 
     |████████████████████████████████| 749 kB 81.0 MB/s 
     |████████████████████████████████| 119 kB 102.5 MB/s 
  Created wheel for cctk: filename=cctk-0.2.12-py3-none-any.whl size=99198 sha256=3d314f3ea916e9af71c69f8f57fea08ac24d7c26fb2cbc8dae8cc5f00b2fdfb1
  Stored in directory: /root/.cache/pip/wheels/de/f6/64/704e186f866f5dd4268c50469194732f95a2b827e00cf6ea15
  Created wheel for presto-md: filename=pr

In [ ]:
import ipywidgets
from ipywidgets import interact, fixed
import warnings
warnings.filterwarnings('ignore')
import py3Dmol

def MolTo3DView(xyz, size=(300, 300), style="stick", surface=False, opacity=0.5):
    """Draw molecule in 3D
    
    Args:
    ----
        mol: rdMol, molecule to show
        size: tuple(int, int), canvas size
        style: str, type of drawing molecule
               style can be 'line', 'stick', 'sphere', 'carton'
        surface, bool, display SAS
        opacity, float, opacity of surface, range 0.0-1.0
    Return:
    ----
        viewer: py3Dmol.view, a class for constructing embedded 3Dmol.js views in ipython notebooks.
    """
    assert style in ('line', 'stick', 'sphere', 'carton')
    # mblock = Chem.MolToMolBlock(mol)
    # xyz = mblock
    viewer = py3Dmol.view(width=size[0], height=size[1])
    viewer.addModel(open(xyz, 'r').read(), 'xyz')
    viewer.setStyle({style:{}})
    viewer.setBackgroundColor('0xeeeeee')

    viewer.setHoverable({},True,'''function(atom,viewer,event,container) {
                   if(!atom.label) {
                    atom.label = viewer.addLabel(atom.atom+":"+atom.serial,{position: atom, backgroundColor: 'mintcream', fontColor:'black'});
                   }}''',
               '''function(atom,viewer) { 
                   if(atom.label) {
                    viewer.removeLabel(atom.label);
                    delete atom.label;
                   }
                }''')


    if surface:
        viewer.addSurface(py3Dmol.SAS, {'opacity': opacity})
    viewer.zoomTo()
    return viewer

In [ ]:
%cd /content/drive/MyDrive/codes/cheapocrest
!chmod 755 *

%cd /content/drive/MyDrive/codes/param_dir/3ob-3-1/
!chmod 755 *
%cd /content/drive/MyDrive/codes/orca/

!chmod 755 *

sys.path.append('/usr/local/lib/python3.7/site-packages/')
sys.path.append("/content/drive/MyDrive/codes/orca")
sys.path.append("/content/drive/MyDrive/codes/param_dir/3ob-3-1/")

ORCA_PATH = '/content/drive/MyDrive/codes/orca'
DFTB_PREFIX = "/content/drive/MyDrive/codes/param_dir/3ob-3-1/"
DFTB_COMMAND = "/usr/local/bin/dftb+"
CHEAPOCREST = "/content/drive/MyDrive/codes/cheapocrest"

os.environ['PATH'] += os.pathsep + ORCA_PATH
os.environ['PATH'] += os.pathsep + DFTB_PREFIX
os.environ['PATH'] += os.pathsep + DFTB_COMMAND
os.environ['PATH'] += os.pathsep + CHEAPOCREST

os.environ['OMPI_ALLOW_RUN_AS_ROOT'] = '1'
os.environ['OMPI_ALLOW_RUN_AS_ROOT_CONFIRM'] = '1'
os.environ['OMPI_MCA_opal_cuda_support'] = 'true'
os.environ['OMPI_MCA_pml'] = "ucx" 
os.environ['OMPI_MCA_osc'] = "ucx"
os.environ['DFTB_PREFIX'] = "/root/.local/dftb/slakos/3ob-3-1"

% cd /content/drive/MyDrive/param_dir
slakos_path = '/root/.local/dftb/slakos'
if not os.path.exists(slakos_path):
	os.makedirs(slakos_path)
! wget -N https://dftb.org/fileadmin/DFTB/public/slako/3ob/3ob-3-1.tar.xz
! wget -N https://dftb.org/fileadmin/DFTB/public/slako/3ob/wfc.3ob-3-1.hsd
! tar -xf 3ob-3-1.tar.xz
! mv 3ob-3-1 /root/.local/dftb/slakos
! mv wfc.3ob-3-1.hsd /root/.local/dftb/slakos/3ob-3-1




/content/drive/MyDrive/codes/cheapocrest
[Errno 2] No such file or directory: '/content/drive/MyDrive/codes/param_dir/3ob-3-1/'
/content/drive/MyDrive/codes/cheapocrest
/content/drive/MyDrive/codes/orca
[Errno 2] No such file or directory: '/content/drive/MyDrive/param_dir'
/content/drive/MyDrive/codes/orca
--2021-12-06 23:20:08--  https://dftb.org/fileadmin/DFTB/public/slako/3ob/3ob-3-1.tar.xz
Resolving dftb.org (dftb.org)... 134.102.38.248, 2001:638:708:1263:1000::20
Connecting to dftb.org (dftb.org)|134.102.38.248|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13072380 (12M) [application/x-xz]
Saving to: ‘3ob-3-1.tar.xz’

3ob-3-1.tar.xz      100%[===================>]  12.47M  7.74MB/s    in 1.6s    

2021-12-06 23:20:10 (7.74 MB/s) - ‘3ob-3-1.tar.xz’ saved [13072380/13072380]

--2021-12-06 23:20:11--  https://dftb.org/fileadmin/DFTB/public/slako/3ob/wfc.3ob-3-1.hsd
Resolving dftb.org (dftb.org)... 134.102.38.248, 2001:638:708:1263:1000::20
Connecting to d

In [ ]:
% cd /content/drive/MyDrive/codes/gap-train
# !conda install --file requirements.txt --channel conda-forge --yes
# !conda install autode==1.0.5 --channel conda-forge --yes
!python setup.py install
import gaptrain as gt

/content/drive/MyDrive/codes/gap-train
running install
running bdist_egg
running egg_info
writing gaptrain.egg-info/PKG-INFO
writing dependency_links to gaptrain.egg-info/dependency_links.txt
writing top-level names to gaptrain.egg-info/top_level.txt
reading manifest file 'gaptrain.egg-info/SOURCES.txt'
adding license file 'LICENSE.md'
writing manifest file 'gaptrain.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/gaptrain
copying build/lib/gaptrain/md.py -> build/bdist.linux-x86_64/egg/gaptrain
copying build/lib/gaptrain/iicalculator.py -> build/bdist.linux-x86_64/egg/gaptrain
copying build/lib/gaptrain/systems.py -> build/bdist.linux-x86_64/egg/gaptrain
copying build/lib/gaptrain/log.py -> build/bdist.linux-x86_64/egg/gaptrain
copying build/lib/gaptrain/molecules.py -> build/bdist.linux-x86_64/egg/gaptrain
copying build/lib/gaptrain/utils.py -

2021-12-06 23:20:56 15da1aeed43c autode.log.log[423] INFO Getting atoms from /content/drive/My Drive/codes/gap-train/gaptrain/solvent_lib/h2o.xyz
2021-12-06 23:20:56 15da1aeed43c gaptrain.log[423] INFO Initialised H2O
Number of atoms      = 3
GROMACS itp filename = /content/drive/My Drive/codes/gap-train/gaptrain/solvent_lib/h2o.itp
2021-12-06 23:20:56 15da1aeed43c autode.log.log[423] INFO Getting atoms from /content/drive/My Drive/codes/gap-train/gaptrain/solvent_lib/tip4p.xyz
2021-12-06 23:20:56 15da1aeed43c gaptrain.log[423] INFO Initialised WH2O
Number of atoms      = 4
GROMACS itp filename = /content/drive/My Drive/codes/gap-train/gaptrain/solvent_lib/tip4p.itp


In [ ]:
% cd /content/drive/MyDrive/codes/gap-train
% mkdir -p gt_test
% cd gt_test


import gaptrain as gt
gt.GTConfig.n_cores = 4

system = gt.System(box_size=[10, 10, 10])
system.add_solvent('h2o', n=20)

training_data = gt.Data()
for i in range(10):
    training_data += system.random()

training_data.parallel_dftb()
gap = gt.GAP(name='random', system=system)
gap.train(training_data)


/content/drive/MyDrive/codes/gap-train


2021-12-06 23:21:38 15da1aeed43c gaptrain.log[423] INFO Initialised a system
Number of molecules = 0
Charge              = 0 e
Spin multiplicity   = 1
2021-12-06 23:21:38 15da1aeed43c gaptrain.log[423] INFO Randomising all 20 molecules in the box
2021-12-06 23:21:38 15da1aeed43c gaptrain.log[423] INFO Randomised all molecules in the system
2021-12-06 23:21:38 15da1aeed43c gaptrain.log[423] INFO Current number of configurations is 1
2021-12-06 23:21:38 15da1aeed43c gaptrain.log[423] INFO Randomising all 20 molecules in the box
2021-12-06 23:21:38 15da1aeed43c gaptrain.log[423] INFO Randomised all molecules in the system
2021-12-06 23:21:38 15da1aeed43c gaptrain.log[423] INFO Current number of configurations is 2
2021-12-06 23:21:38 15da1aeed43c gaptrain.log[423] INFO Randomising all 20 molecules in the box
2021-12-06 23:21:38 15da1aeed43c gaptrain.log[423] INFO Randomised all molecules in the system
2021-12-06 23:21:38 15da1aeed43c gaptrain.log[423] INFO Current number of configurations

/content/drive/MyDrive/codes/gap-train/gt_test


2021-12-06 23:21:38 15da1aeed43c gaptrain.log[423] INFO Randomised all molecules in the system
2021-12-06 23:21:38 15da1aeed43c gaptrain.log[423] INFO Current number of configurations is 4
2021-12-06 23:21:38 15da1aeed43c gaptrain.log[423] INFO Randomising all 20 molecules in the box
2021-12-06 23:21:38 15da1aeed43c gaptrain.log[423] INFO Randomised all molecules in the system
2021-12-06 23:21:38 15da1aeed43c gaptrain.log[423] INFO Current number of configurations is 5
2021-12-06 23:21:38 15da1aeed43c gaptrain.log[423] INFO Randomising all 20 molecules in the box
2021-12-06 23:21:38 15da1aeed43c gaptrain.log[423] INFO Randomised all molecules in the system
2021-12-06 23:21:38 15da1aeed43c gaptrain.log[423] INFO Current number of configurations is 6
2021-12-06 23:21:38 15da1aeed43c gaptrain.log[423] INFO Randomising all 20 molecules in the box
2021-12-06 23:21:38 15da1aeed43c gaptrain.log[423] INFO Randomised all molecules in the system
2021-12-06 23:21:38 15da1aeed43c gaptrain.log[423]

Running calculations over 4 cores


2021-12-06 23:21:39 15da1aeed43c gaptrain.log[423] INFO Calculations done in 0.0 m
2021-12-06 23:21:39 15da1aeed43c gaptrain.log[423] WARNING Not adding SOAP on H
2021-12-06 23:21:39 15da1aeed43c gaptrain.log[423] INFO Training a Gaussian Approximation potential on *10* training data points
2021-12-06 23:21:39 15da1aeed43c gaptrain.log[423] INFO Saving 10 configurations to data.xyz
2021-12-06 23:21:39 15da1aeed43c gaptrain.log[423] INFO Adding SOAP:              O
2021-12-06 23:21:39 15da1aeed43c gaptrain.log[423] INFO with neighbours           ['H', 'O']


GAP training ran in 0.0 m


In [ ]:
h2o = gt.System(box_size=[10, 10, 10])
h2o.add_solvent('h2o', n=20)

2021-12-06 23:21:50 15da1aeed43c gaptrain.log[423] INFO Initialised a system
Number of molecules = 0
Charge              = 0 e
Spin multiplicity   = 1


In [ ]:
data, gap = gt.active.train_ii(h2o, method_name="xtb")

2021-12-06 23:21:56 15da1aeed43c gaptrain.log[423] INFO Initialised a system
Number of molecules = 0
Charge              = 0 e
Spin multiplicity   = 1
2021-12-06 23:21:56 15da1aeed43c gaptrain.log[423] WARNING Not adding SOAP on H
2021-12-06 23:21:56 15da1aeed43c gaptrain.log[423] INFO Randomising all 1 molecules in the box
2021-12-06 23:21:56 15da1aeed43c gaptrain.log[423] INFO Randomised all molecules in the system
2021-12-06 23:21:56 15da1aeed43c gaptrain.log[423] INFO Randomising all 1 molecules in the box
2021-12-06 23:21:56 15da1aeed43c gaptrain.log[423] INFO Randomised all molecules in the system
2021-12-06 23:21:56 15da1aeed43c gaptrain.log[423] INFO Randomising all 1 molecules in the box
2021-12-06 23:21:56 15da1aeed43c gaptrain.log[423] INFO Randomised all molecules in the system
2021-12-06 23:21:56 15da1aeed43c gaptrain.log[423] INFO Randomising all 1 molecules in the box
2021-12-06 23:21:56 15da1aeed43c gaptrain.log[423] INFO Randomised all molecules in the system
2021-12-0

Running calculations over 4 cores


2021-12-06 23:21:56 15da1aeed43c autode.log.log[652] INFO Calculation is in the gas phase
2021-12-06 23:21:56 15da1aeed43c autode.log.log[653] INFO Calculation is in the gas phase
2021-12-06 23:21:56 15da1aeed43c autode.log.log[652] INFO Running calculation tmp_xtb
2021-12-06 23:21:56 15da1aeed43c autode.log.log[654] INFO Calculation is in the gas phase
2021-12-06 23:21:56 15da1aeed43c autode.log.log[651] INFO Calculation is in the gas phase
2021-12-06 23:21:56 15da1aeed43c autode.log.log[652] INFO Generating input file(s) for tmp_xtb
2021-12-06 23:21:56 15da1aeed43c autode.log.log[654] INFO Running calculation tmp_xtb
2021-12-06 23:21:56 15da1aeed43c autode.log.log[652] INFO No calculations have been performed here yet
2021-12-06 23:21:56 15da1aeed43c autode.log.log[653] INFO Running calculation tmp_xtb
2021-12-06 23:21:56 15da1aeed43c autode.log.log[651] INFO Running calculation tmp_xtb
2021-12-06 23:21:56 15da1aeed43c autode.log.log[653] INFO Generating input file(s) for tmp_xtb
202

GAP training ran in 0.0 m


2021-12-06 23:21:58 15da1aeed43c gaptrain.log[1149] INFO Running GAP MD
2021-12-06 23:21:58 15da1aeed43c gaptrain.log[1152] INFO Running GAP MD
2021-12-06 23:21:58 15da1aeed43c gaptrain.log[1149] INFO Running 4 steps with a timestep of 0.5 fs
2021-12-06 23:21:58 15da1aeed43c gaptrain.log[1149] INFO Using 1 cores for GAP MD
2021-12-06 23:21:58 15da1aeed43c gaptrain.log[1152] INFO Running 4 steps with a timestep of 0.5 fs
2021-12-06 23:21:58 15da1aeed43c gaptrain.log[1149] INFO Initialising initial velocities for a temperature 1000.0 K
2021-12-06 23:21:58 15da1aeed43c gaptrain.log[1152] INFO Using 1 cores for GAP MD
2021-12-06 23:21:58 15da1aeed43c gaptrain.log[1151] INFO Running GAP MD
2021-12-06 23:21:58 15da1aeed43c gaptrain.log[1150] INFO Running GAP MD
2021-12-06 23:21:58 15da1aeed43c gaptrain.log[1152] INFO Initialising initial velocities for a temperature 1000.0 K
2021-12-06 23:21:58 15da1aeed43c gaptrain.log[1150] INFO Running 4 steps with a timestep of 0.5 fs
2021-12-06 23:21:58

...

Running calculations over 4 cores


2021-12-06 23:36:10 15da1aeed43c gaptrain.log[423] INFO Calculations done in 0.0 m
2021-12-06 23:36:10 15da1aeed43c gaptrain.log[423] INFO Current number of configurations is 40
2021-12-06 23:36:10 15da1aeed43c gaptrain.log[423] INFO Training the intermolecular component of the potential. Expecting data that is free from intra energy and force
2021-12-06 23:36:10 15da1aeed43c gaptrain.log[423] INFO Training a Gaussian Approximation potential on *40* training data points
2021-12-06 23:36:10 15da1aeed43c gaptrain.log[423] INFO Saving 40 configurations to intra_h2o_inter_h2o.xyz
2021-12-06 23:36:10 15da1aeed43c gaptrain.log[423] INFO Adding SOAP:              O
2021-12-06 23:36:10 15da1aeed43c gaptrain.log[423] INFO with neighbours           ['H', 'O']
2021-12-06 23:36:19 15da1aeed43c gaptrain.log[423] INFO Searching for "active" configurations with a threshold of 0.867260 eV
2021-12-06 23:36:19 15da1aeed43c gaptrain.log[423] INFO Using 4 processes


GAP training ran in 0.2 m


2021-12-06 23:36:19 15da1aeed43c gaptrain.log[28794] INFO Running an extra 115.0 fs of MD before calculating an error
2021-12-06 23:36:19 15da1aeed43c gaptrain.log[28794] INFO Running GAP MD
2021-12-06 23:36:19 15da1aeed43c gaptrain.log[28795] INFO Running an extra 115.0 fs of MD before calculating an error
2021-12-06 23:36:19 15da1aeed43c gaptrain.log[28796] INFO Running an extra 115.0 fs of MD before calculating an error
2021-12-06 23:36:19 15da1aeed43c gaptrain.log[28794] INFO Running 234 steps with a timestep of 0.5 fs
2021-12-06 23:36:19 15da1aeed43c gaptrain.log[28794] INFO Using 1 cores for GAP MD
2021-12-06 23:36:19 15da1aeed43c gaptrain.log[28794] INFO Initialising initial velocities for a temperature 300.0 K
2021-12-06 23:36:19 15da1aeed43c gaptrain.log[28797] INFO Running an extra 115.0 fs of MD before calculating an error
2021-12-06 23:36:19 15da1aeed43c gaptrain.log[28795] INFO Running GAP MD
2021-12-06 23:36:19 15da1aeed43c gaptrain.log[28796] INFO Running GAP MD
2021-12-

KeyboardInterrupt: ignored

In [ ]:
traj = gt.md.run_gapmd(configuration=h2o.random(),
                       gap=gap,
                       temp=300,     # Kelvin
                       dt=0.5,       # femtoseconds
                       interval=5,   # frames
                       ps=1,         # picoseconds
                       n_cores=4)

traj.save(filename='traj.xyz')

2021-12-06 23:45:20 15da1aeed43c gaptrain.log[423] INFO Randomising all 20 molecules in the box
2021-12-06 23:45:20 15da1aeed43c gaptrain.log[423] INFO Randomised all molecules in the system
2021-12-06 23:45:20 15da1aeed43c gaptrain.log[423] INFO Running GAP MD
2021-12-06 23:45:20 15da1aeed43c gaptrain.log[423] INFO Running 2000 steps with a timestep of 0.5 fs
2021-12-06 23:45:20 15da1aeed43c gaptrain.log[423] INFO Using 4 cores for GAP MD
2021-12-06 23:45:20 15da1aeed43c gaptrain.log[423] INFO Initialising initial velocities for a temperature 300 K


KeyboardInterrupt: ignored

In [ ]:
% mkdir -p /content/drive/MyDrive/autode_test
% cd /content/drive/MyDrive/autode_test

import autode as ade
ade.Config.n_cores = 4
ade.Config.ORCA.keywords.set_functional('PBE')

rxn = ade.Reaction('C=CC=C.C=C>>C1=CCCCC1')
rxn.locate_transition_state()
rxn.ts.print_xyz_file(filename='ts.xyz')


In [ ]:
import gaptrain as gt
from autode.wrappers.keywords import GradientKeywords

gt.GTConfig.orca_keywords = GradientKeywords(['PBE', 'def2-SVP', 'EnGrad'])

da_ts = gt.System(box_size=[10, 10, 10])
da_ts.add_molecules(gt.Molecule('ts.xyz'))

data, gap = gt.active.train(da_ts,
                            method_name='orca',
                            temp=500,                 # Kelvin
                            active_e_thresh=0.1,      # eV
                            max_time_active_fs=200,   # femtoseconds
                            fix_init_config=True)


In [ ]:
gt.GTConfig.orca_keywords = GradientKeywords(['RI-MP2', 'def2-TZVP', 'TightSCF',
                                              'AutoAux', 'NoFrozenCore', 'EnGrad'])
data.parallel_orca()

gap.train(data)


In [ ]:
traj = gt.md.run_gapmd(configuration=da_ts.random(),
                       gap=gap,
                       temp=50,      # Kelvin
                       dt=0.5,       # femtoseconds
                       interval=1,   # frames
                       fs=400,       # femtoseconds
                       n_cores=4)

traj.save('da_mp2_traj.xyz')


In [ ]:
% mkdir -p /content/drive/MyDrive/bullvalene_test
% cd /content/drive/MyDrive/bullvalene_test


import gaptrain as gt
import autode as ade
from autode.wrappers.keywords import GradientKeywords, OptKeywords

ade.Config.n_cores = 4
gt.GTConfig.n_cores = 4
gt.GTConfig.orca_keywords = GradientKeywords(['PBE', 'def2-SVP', 'D3BJ', 'EnGrad'])

# Generate an optimised bullvalene molecule
mol = ade.Molecule(smiles='C12C=C[C@H]3[C@@H](C=C2)[C@H]3C=C1', name='bullvalene')
mol.optimise(method=ade.methods.ORCA(),
             keywords=OptKeywords(['PBE', 'def2-SVP', 'D3BJ', 'Looseopt']))
mol.print_xyz_file()

# Create a gap-train system and train
system = gt.System(box_size=[10, 10, 10])
system.add_molecules(gt.Molecule('bullvalene.xyz'))



In [ ]:
% mkdir -p /content/drive/MyDrive/bullvalene_test
% cd /content/drive/MyDrive/bullvalene_test

/content/drive/MyDrive/bullvalene_test


In [ ]:
MolTo3DView("bullvalene.xyz")

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
data, gap = gt.active.train(system,
                            method_name='orca',
                            temp=500,
                            max_time_active_fs=200,
                            validate=False,
                            bbond_energy={(3, 4): 2},  # Atoms indexed from 0..
                            max_active_iters=15)       # ..and the energy in eV



In [ ]:
# Run 1 ps molecular dynamics using the GAP at 300 K using a 0.5 fs time-step.
# The initial configuration is methane located at a random position in the box
traj = gt.md.run_gapmd(configuration=system.random(),
                       gap=gap,
                       temp=500,           # Kelvin
                       dt=0.5,             # fs
                       interval=5,         # frames
                       ps=100,
                       n_cores=4)

# and save the trajectory for visualisation in Avogadro, VMD, PyMol etc.
traj.save(filename='traj_longer.xyz')